In [15]:
import tensorflow as tf
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np


X, y = make_regression(n_samples=60, n_features=10, noise=0.5, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
class Layer:
    def __init__(self, units, input_layer=False):
        self.input_layer = input_layer
        self.units = units
        self.activation = 'relu'
        self.grad_w = None
        self.grad_b = None

    def _setGrad(self, w, b):
        self.grad_w = w
        self.grad_b = b

    def _updateGrad(self):
        self.w.assign_sub(0.0000001 * self.grad_w)
        self.bias.assign_sub(0.0000001 * self.grad_b)

    def _weightInit(self, input_size):
        self.w = tf.Variable(tf.random.normal((input_size, self.units), mean = 0, stddev = 1 / input_size))
        self.bias = tf.Variable(tf.zeros((1, self.units)))

    @tf.function
    def __call__(self, X):
        return X @ self.w + self.bias

In [17]:
class DenseNetwork:
    def __init__(self, layers):
        self.layers = layers

        for i in range(1, len(self.layers)):
            self.layers[i]._weightInit(self.layers[i - 1].units)


    def fit(self, X, y):
        X = tf.constant(X, dtype="float32")
        y = tf.constant(y, dtype="float32")
        
        for _ in range(1000):
            with tf.GradientTape(persistent=True) as tape:
                loss = self.network(X, y)

            for layer in self.layers[1:]:
                layer._setGrad(tape.gradient(loss, layer.w), tape.gradient(loss, layer.bias))

            for layer in self.layers[1:]:
                layer._updateGrad()
                
        del tape

    def predict(self, X):
        X = tf.constant(X, dtype='float32')
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    def network(self, X, y):
        y_pred = self.forward(X)
        loss = tf.losses.mean_squared_error(y_pred, y)
        # tf.print(loss)
        return loss

    def forward(self, X):
        for layer in self.layers[1:]:
            X = layer(X)
        return X

In [19]:
nn = DenseNetwork([Layer(10), Layer(30),  Layer(1)])

nn.fit(X_train_scaled, y_train)

nn.predict(X_train_scaled)

<tf.Tensor: shape=(48, 1), dtype=float32, numpy=
array([[ 3.976597  ],
       [ 3.351717  ],
       [-3.030743  ],
       [ 1.5622501 ],
       [ 2.2012546 ],
       [ 0.98106563],
       [ 2.6035233 ],
       [ 5.693296  ],
       [-7.436695  ],
       [ 3.5384905 ],
       [ 1.2381015 ],
       [-5.2438507 ],
       [ 0.80521876],
       [-2.9852412 ],
       [ 0.79733264],
       [-2.3618028 ],
       [ 4.5143757 ],
       [ 6.6734104 ],
       [ 1.9951106 ],
       [ 0.92633355],
       [-1.3621446 ],
       [-0.21452245],
       [-1.5702977 ],
       [-1.989258  ],
       [ 6.1342587 ],
       [-1.1304553 ],
       [-3.435818  ],
       [-7.2537484 ],
       [-1.5377839 ],
       [ 6.8089576 ],
       [ 0.8809441 ],
       [ 1.1289088 ],
       [-3.3047817 ],
       [-5.1496515 ],
       [ 1.7780814 ],
       [-4.2405567 ],
       [-3.5647845 ],
       [-2.7644114 ],
       [ 0.4522092 ],
       [ 3.4599798 ],
       [-6.92695   ],
       [-0.4469901 ],
       [ 2.2081656 ],
     